In [27]:
using Random
using StatsBase
using Formatting: printfmt


In [2]:
function parse_file(file)
    instancia = open("instancias/"*file)
    arr  = []
    for line in eachline(instancia)
        push!(arr, line)
    end
    close(instancia)
    
    vertices = []
    premios = []
    flag_vertices = true
    for line in arr[8:length(arr)]
        if startswith(line, "DEMAND_SECTION")
            flag_vertices = false
        elseif startswith(line, "DEPOT_SECTION")
            break
        elseif flag_vertices
            x, y = split(line)[2:3]
            x = parse(Int32, x)
            y = parse(Int32, y)
            push!(vertices, (x,y))
        else
            premio = split(line)[2]
            premio = parse(Int32, premio)
            push!(premios, premio)

        end
    end
    return (vertices, premios)
end


function get_distancias(vertices)
    cardinalidade = length(vertices)
    distancias = zeros(cardinalidade, cardinalidade)
    for i in 1:cardinalidade
        for j in 1:cardinalidade
            distancia = ((vertices[i][1] - vertices[j][1])^2 + (vertices[i][2] - vertices[j][2])^2)^(0.5)
            distancias[i, j] =  round(distancia)
        end
    end
    return distancias
end

get_distancias (generic function with 1 method)

In [3]:
arquivo = "A-n32-k5.vrp"
v, P = parse_file(arquivo)
D = get_distancias(v)
k = length(P);

k=8
D = D[1:k, 1:k]
P = P[1:k];

In [12]:
function get_solucao_inicial(k)
    solucao_inicial = [1]
    vertices_nao_visitados = collect(2:k)
#     for i in 1:k-1
#         solucao_inicial[i, i+1] = 1
#     end
#     solucao_inicial[k, 1] = 1
    return solucao_inicial, vertices_nao_visitados
end

function get_tabela_tabu_inicial(k)
    return zeros(Bool, k, k)
end

get_tabela_tabu_inicial (generic function with 1 method)

In [13]:
solucao, vertices_nao_visitados  = get_solucao_inicial(k);

In [215]:
solucao = [1, 3, 7, 4]
vertices_nao_visitados = [2, 5, 6, 8]

4-element Array{Int64,1}:
 2
 5
 6
 8

In [126]:
tabela_tabu = get_tabela_tabu_inicial(k);

In [93]:
# adicionar
vertice_nao_visitado = vertices_nao_visitados[rand(1:length(vertices_nao_visitados))]
indice_de_inclusao = rand(2:length(solucao)+1)
direcao = 1
printfmt("$funcao, $indice_de_inclusao, $vertice_nao_visitado")

1, 2, 4

In [31]:
printfmt("$funcao, $indice_de_inclusao, $vertice_nao_visitado")

1, 4, 6

In [129]:
# remover
if length(solucao) == 1
    indice_de_remocao = 0
else
    indice_de_remocao = rand(2:length(solucao))
end
direcao = 0
printfmt("$direcao, $indice_de_remocao")

0, 3

In [374]:
function pop_at(array, index)
    valor = array[index]
    deleteat!(array, index)
    return array, valor
end

pop_at (generic function with 1 method)

In [388]:
function get_um_vizinho_a_direita(solucao, vertices_nao_visitados)
    direcao = 1
    
    # copias de seguranca
    vizinho = copy(solucao)
    copia_vertices_nao_visitados = copy(vertices_nao_visitados)
    
    # escolhe um vertice para colocar no caminho
    indice = rand(1:length(copia_vertices_nao_visitados))
    vertice_adicionado = copia_vertices_nao_visitados[indice]
    
    # deleta o vertice do array de vertices nao visitados
    deleteat!(copia_vertices_nao_visitados, indice)
    
    # escolhe um ponto de insercao do vertice na solucao
    indice_de_inclusao = rand(2:length(solucao)+1)
    
    # insere o vertice na solucao
    insert!(vizinho, indice_de_inclusao, vertice_adicionado)
    
    # retorna vizinho, vertices nao visitados, tripla da tabela tabu
    return vizinho, copia_vertices_nao_visitados, (direcao, vertice_adicionado, indice_de_inclusao)
end
    

get_um_vizinho_a_direita (generic function with 2 methods)

In [390]:
function get_um_vizinho_a_esquerda(solucao, vertices_nao_visitados)
    direcao = 0
    
    # copias de seguranca
    vizinho = copy(solucao)
    copia_vertices_nao_visitados = copy(vertices_nao_visitados)
    
    # escolhe o indice do vertice no caminho para ser removido
    indice_de_exclusao = rand(2:length(solucao))
    
    # remove o vertice escolhido
    vizinho, vertice_deletado = pop_at(vizinho, indice_de_exclusao)
    
    # insere o vertice removido do caminho na array de vertices nao visitados
    push!(copia_vertices_nao_visitados, vertice_deletado);
    
    # retorna vizinho, vertices nao visitados, tripla da tabela tabu
    return vizinho, copia_vertices_nao_visitados, (direcao, vertice_deletado, 0)
end

get_um_vizinho_a_esquerda (generic function with 1 method)

In [285]:
function get_custo(caminho)
    custo = 0
    for i in 1:length(caminho)-1
        saida = caminho[i]
        entrada = caminho[i+1]
        custo += D[saida, entrada] - P[entrada]
    end
    saida = last(caminho)
    entrada = 1
    custo += D[saida, entrada] - P[entrada] 
end

get_custo (generic function with 1 method)

In [398]:
solucao = [1, 3, 7, 4]
vertices_nao_visitados = [2, 5, 6, 8]

get_um_vizinho_a_direita(solucao, vertices_nao_visitados)

([1, 6, 3, 7, 4], [2, 5, 8], (1, 6, 2))

In [397]:
solucao = [1, 3, 7, 4]
vertices_nao_visitados = [2, 5, 6, 8]

get_um_vizinho_a_esquerda(solucao, vertices_nao_visitados)

([1, 7, 4], [2, 5, 6, 8, 3], (0, 3, 0))

In [392]:
vertices_nao_visitados

5-element Array{Int64,1}:
 2
 5
 6
 8
 3

In [309]:
tupla_vizinho_custo

5-element Array{Any,1}:
 ([1, 3, 7, 2, 4], 221.0)
 ([1, 6, 3, 7, 4], 222.0)
 ([1, 3, 8, 7, 4], 199.0)
 ([1, 3, 2, 7, 4], 220.0)
 ([1, 3, 2, 7, 4], 220.0)

In [157]:
vertices_nao_visitados[rand(1:length(vertices_nao_visitados))]

3

In [143]:
vertice_nao_visitados

4-element Array{Int64,1}:
 2
 5
 6
 8

In [167]:
insert!([6, 5, 4, 2, 1], 4, 9)

6-element Array{Int64,1}:
 6
 5
 4
 9
 2
 1

In [369]:
A = [1, 2, 3, 4]
a = pop!(A)

4

In [371]:
function pop_at(array, index)
    valor = array[index]
    deleteat!(array, index)
    return array, valor
end

pop_at (generic function with 1 method)

In [373]:
A = [1, 2, 3, 4]
pop_at(A, 3)

([1, 2, 4], 3)

In [367]:
A

3-element Array{Int64,1}:
 1
 3
 4